In [1]:

"""
Week 2 — BTS On-Time: Cleaning & Feature Engineering (Plan B: PyArrow)

Inputs (expected under --root):
- eda/sample_100k.parquet                  # preferred quick-run source
- parquet/bts_on_time_all.parquet          # fallback if sample not found

Outputs (under --root):
- eda/sample_100k_week2_features.parquet
- eda/feature_dictionary_week2.csv

Features created:
- Cleaned columns (drop Unnamed:*), Arrow-friendly string casts
- Time normalization: HHMM → minutes since midnight (CRS/actual dep/arr)
- Calendar: dow, weekend, month, quarter, season, time-of-day bin
- (Optional) US federal holiday flags if 'holidays' is installed
- Profiles (joined back to each row):
    * route_*  : Origin–Dest median ArrDelay, on-time rate, counts
    * carrier_*: carrier median ArrDelay, on-time rate, counts
"""

import argparse
from pathlib import Path
import numpy as np
import pandas as pd

# --------------------------
# CLI
# --------------------------
def parse_args(argv=None):
    p = argparse.ArgumentParser(description="Week 2 cleaning & feature engineering")
    p.add_argument("--root", type=str, default="./bts_on_time_data",
                   help="Project root containing eda/ and parquet/")
    p.add_argument("--write-full", action="store_true",
                   help="Also write a full-size features parquet if reading from all.parquet")
    return p.parse_known_args(argv)[0]

# --------------------------
# Utilities
# --------------------------
def hhmm_to_minutes(x):
    """Convert HHMM integer/string to minutes since midnight (0..1439)."""
    if pd.isna(x):
        return np.nan
    try:
        v = int(x)
        if v < 0:
            return np.nan
        hh, mm = divmod(v, 100)
        if (0 <= hh <= 24) and (0 <= mm <= 59):
            total = 60 * hh + mm
            # BTS sometimes encodes midnight as 2400 → map to 0
            return 0 if total == 1440 else total
        return np.nan
    except Exception:
        return np.nan

def month_to_season(m):
    """Meteorological seasons: DJF, MAM, JJA, SON."""
    if m in (12, 1, 2):  return "DJF"
    if m in (3, 4, 5):   return "MAM"
    if m in (6, 7, 8):   return "JJA"
    if m in (9, 10, 11): return "SON"
    return pd.NA

def tod_bin(mins):
    """Time-of-day bins from minutes since midnight."""
    if pd.isna(mins): return pd.NA
    mins = float(mins)
    # 6 bins: Night [0,6), Early [6,9), Morning [9,12), Midday [12,15), Evening [15,19), Late [19,24)
    if 0   <= mins < 360:   return "night"
    if 360 <= mins < 540:   return "early"
    if 540 <= mins < 720:   return "morning"
    if 720 <= mins < 900:   return "midday"
    if 900 <= mins < 1140:  return "evening"
    if 1140<= mins < 1440:  return "late"
    return pd.NA

def safe_carrier_col(df):
    if "Reporting_Airline" in df.columns:
        return "Reporting_Airline"
    if "IATA_CODE_Reporting_Airline" in df.columns:
        return "IATA_CODE_Reporting_Airline"
    # fallback to a likely legacy name
    if "UniqueCarrier" in df.columns:
        return "UniqueCarrier"
    raise KeyError("Carrier column not found")

def add_us_holidays(df, date_col="FlightDate"):
    """Attach US federal holiday flags if 'holidays' is installed; otherwise no-op."""
    try:
        import holidays
        us_holidays = holidays.US()
        col = "is_us_holiday"
        df[col] = df[date_col].dt.date.map(lambda d: d in us_holidays if pd.notna(d) else False)
        return [col]
    except Exception:
        # library not installed or other issue; skip silently
        return []

# --------------------------
# Profiles
# --------------------------
def build_route_profiles(df):
    """Origin-Dest profiles: median arrival delay, on-time rate, flight counts."""
    base = df[["Origin", "Dest", "ArrDelay", "ArrDel15"]].copy()
    # robust to missing ArrDel15 (should be 0/1)
    base["ArrDel15"] = pd.to_numeric(base["ArrDel15"], errors="coerce")
    gp = (base
          .groupby(["Origin","Dest"], dropna=False)
          .agg(route_med_arr_delay=("ArrDelay","median"),
               route_ontime_rate=("ArrDel15", lambda s: 1 - np.nanmean(s)),
               route_flights=("ArrDel15","count"))
          .reset_index())
    return gp

def build_carrier_profiles(df, carrier_col):
    base = df[[carrier_col, "ArrDelay", "ArrDel15"]].copy()
    base["ArrDel15"] = pd.to_numeric(base["ArrDel15"], errors="coerce")
    gp = (base
          .groupby([carrier_col], dropna=False)
          .agg(carrier_med_arr_delay=("ArrDelay","median"),
               carrier_ontime_rate=("ArrDel15", lambda s: 1 - np.nanmean(s)),
               carrier_flights=("ArrDel15","count"))
          .reset_index())
    return gp

# --------------------------
# Main feature builder
# --------------------------
def main(argv=None):
    args = parse_args(argv)
    ROOT = Path(args.root)
    EDA_DIR = ROOT / "eda"
    PARQ_DIR = ROOT / "parquet"

    EDA_DIR.mkdir(parents=True, exist_ok=True)

    # Prefer the 100k sample for Week 2 dev speed; fallback to full
    sample_path = EDA_DIR / "sample_100k.parquet"
    full_path   = PARQ_DIR / "bts_on_time_all.parquet"

    if sample_path.exists():
        src = sample_path
        mode = "sample"
    elif full_path.exists():
        src = full_path
        mode = "full"
    else:
        raise FileNotFoundError("Neither eda/sample_100k.parquet nor parquet/bts_on_time_all.parquet exists.")

    print(f"[LOAD] {src}")
    df = pd.read_parquet(src)

    # -------- Cleaning & type normalization --------
    # drop junk tail columns
    junk = [c for c in df.columns if str(c).startswith("Unnamed:")]
    if junk:
        df = df.drop(columns=junk, errors="ignore")
        print(f"[CLEAN] Dropped junk columns: {junk}")

    # cast object-ish columns to Arrow-friendly string; leave numerics/booleans as is
    for c in df.columns:
        if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_string_dtype(df[c]):
            df[c] = df[c].astype("string")

    # Ensure key columns exist
    must_have = ["FlightDate","Year","Month","Origin","Dest","ArrDelay","DepDelay"]
    for col in must_have:
        if col not in df.columns:
            raise KeyError(f"Missing required column: {col}")

    # parse FlightDate → datetime (robust)
    df["FlightDate"] = pd.to_datetime(df["FlightDate"], errors="coerce")

    # -------- Time normalization (HHMM → minutes) --------
    for tcol in ["CRSDepTime","DepTime","CRSArrTime","ArrTime"]:
        if tcol in df.columns:
            df[f"{tcol}_min"] = df[tcol].map(hhmm_to_minutes)

    # time-of-day bins from scheduled times (prefer CRSDepTime)
    ref_col = "CRSDepTime_min" if "CRSDepTime_min" in df.columns else None
    if ref_col:
        df["tod_bin"] = df[ref_col].map(tod_bin).astype("string")

    # -------- Calendar features --------
    df["dow"]       = df["FlightDate"].dt.dayofweek   # 0=Mon
    df["is_weekend"]= df["dow"].isin([5,6])
    df["quarter"]   = df["FlightDate"].dt.quarter
    df["season"]    = df["Month"].map(month_to_season).astype("string")

    # optional: US holiday flag
    added = add_us_holidays(df, "FlightDate")
    if added:
        print(f"[FE] Added holiday flags: {added}")

    # -------- Profiles (route & carrier) --------
    carrier_col = safe_carrier_col(df)
    # make sure numeric
    for col in ["ArrDelay","ArrDel15"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    route_prof   = build_route_profiles(df)
    carrier_prof = build_carrier_profiles(df, carrier_col)

    # merge back
    df = df.merge(route_prof,   on=["Origin","Dest"], how="left")
    df = df.merge(carrier_prof, on=[carrier_col],    how="left")

    # -------- Save outputs --------
    out_feat = EDA_DIR / ("sample_100k_week2_features.parquet" if mode=="sample" else "bts_on_time_week2_features.parquet")
    df.to_parquet(out_feat, index=False)
    print(f"[SAVE] Feature table → {out_feat.resolve()} (rows={len(df)}, cols={df.shape[1]})")

    # data dictionary (simple)
    dictionary_rows = []
    def add_dict(name, dtype, desc):
        dictionary_rows.append({"column": name, "dtype": str(dtype), "description": desc})

    created = [
        ("CRSDepTime_min","minutes since midnight derived from HHMM"),
        ("DepTime_min","minutes since midnight derived from HHMM"),
        ("CRSArrTime_min","minutes since midnight derived from HHMM"),
        ("ArrTime_min","minutes since midnight derived from HHMM"),
        ("tod_bin","time-of-day bin from CRSDepTime_min (night/early/morning/midday/evening/late)"),
        ("dow","day of week (0=Mon)"),
        ("is_weekend","bool"),
        ("quarter","calendar quarter 1–4"),
        ("season","meteorological season (DJF/MAM/JJA/SON)"),
        ("route_med_arr_delay","median ArrDelay for Origin–Dest"),
        ("route_ontime_rate","1 - mean(ArrDel15) for Origin–Dest"),
        ("route_flights","count for Origin–Dest"),
        ("carrier_med_arr_delay","median ArrDelay for carrier"),
        ("carrier_ontime_rate","1 - mean(ArrDel15) for carrier"),
        ("carrier_flights","count for carrier"),
    ]
    for name, desc in created:
        if name in df.columns:
            add_dict(name, df[name].dtype, desc)

    # also record junk columns that were dropped (for transparency)
    for c in junk:
        add_dict(c, "dropped", "dropped junk/empty tail column")

    dict_path = EDA_DIR / "feature_dictionary_week2.csv"
    pd.DataFrame(dictionary_rows).to_csv(dict_path, index=False)
    print(f"[SAVE] Data dictionary → {dict_path.resolve()}")

    # optionally also write full-size features if requested AND src was full
    if args.write_full and mode == "full":
        full_out = PARQ_DIR / "bts_on_time_week2_features.parquet"
        df.to_parquet(full_out, index=False)
        print(f"[SAVE] Full features → {full_out.resolve()}")

    # quick preview
    preview_cols = [c for c in [
        "FlightDate","Origin","Dest",carrier_col,
        "ArrDelay","DepDelay","Distance","TaxiOut",
        "CRSDepTime_min","tod_bin","dow","is_weekend","season",
        "route_med_arr_delay","route_ontime_rate",
        "carrier_med_arr_delay","carrier_ontime_rate"
    ] if c in df.columns][:16]
    print("\n[PREVIEW]")
    print(df[preview_cols].head(10))

if __name__ == "__main__":
    import sys
    main(sys.argv[1:])

[LOAD] bts_on_time_data/eda/sample_100k.parquet
[CLEAN] Dropped junk columns: ['Unnamed: 109']


/var/folders/vf/3hrzq4rn5dvd6nlv_pfk814m0000gn/T/ipykernel_91336/2461556558.py:114: RuntimeWarning: Mean of empty slice
  route_ontime_rate=("ArrDel15", lambda s: 1 - np.nanmean(s)),


[SAVE] Feature table → /Users/guohaoyang/Desktop/vscworkspace/BTS/bts_on_time_data/eda/sample_100k_week2_features.parquet (rows=100000, cols=124)
[SAVE] Data dictionary → /Users/guohaoyang/Desktop/vscworkspace/BTS/bts_on_time_data/eda/feature_dictionary_week2.csv

[PREVIEW]
  FlightDate Origin Dest Reporting_Airline  ArrDelay  DepDelay  Distance  \
0 2025-04-01    MCI  DFW                AA     -19.0      -7.0     460.0   
1 2024-01-08    DTW  GRR                DL     -25.0      -8.0     120.0   
2 2024-03-27    DEN  SFO                UA      51.0      34.0     967.0   
3 2024-10-09    FLL  DCA                B6     -24.0      -6.0     899.0   
4 2024-12-11    PHL  BNA                WN       5.0      11.0     675.0   
5 2025-04-07    FLL  ATL                DL      98.0     104.0     581.0   
6 2025-03-02    PDX  FLL                AS     -21.0      -5.0    2694.0   
7 2024-11-08    ORD  IND                YX      -9.0       0.0     177.0   
8 2025-01-10    DCA  ORD                U